In [ ]:
import sys
from nlputils import utils
from nlputils.components.pymupdf_util import pymuprocessor
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import json
import os

c:\Users\Prashant\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load docs

In [ ]:
# Define the path var to dir which need to be processed
path_to_orig_data_folder = "folder_location"

# Define file types to be considered
file_types = ['pdf','docx']

# read all files (inlcuding sub-dir) in dir defined above
files_list = utils.get_files(path_to_orig_data_folder)
print('individuals file type counts', '\n'.join([str(type)+":"+str(len(lst)) 
                                    for type,lst in files_list.items()]), sep='\n')

# Contructing dataframe for record keeping
# create list of key, values for dataframe
keys = [k for k, v in files_list.items() for _ in v]
values = [v for v in files_list.values() for v in v]
df_files = pd.DataFrame({'filetype': keys, 'orig_filepath': values})

individuals file type counts
pdf:5
docx:0


In [3]:
# check if pdf is normal or imagepdf
df_files['ImagePDF'] = df_files.progress_apply(lambda x: utils.check_if_imagepdf(x['orig_filepath']) 
                                                if x['filetype']  == 'pdf' else
                                                (utils.check_if_imagepdf(x['converted_filepath']) 
                                                 if x['converted_filepath'] is not None else None),
                                                 axis=1)
df_files['filename'] = df_files.orig_filepath.apply(lambda x: os.path.basename(x))
df_files['page_count'] =df_files.orig_filepath.apply(lambda x: utils.get_page_count(x))

100%|██████████| 5/5 [00:00<00:00,  9.45it/s]


In [ ]:
# save the files info
path_to_new_folder = "savetofolderlocation"
if not os.path.exists(path_to_new_folder + 'tmp/'):
            os.makedirs(path_to_new_folder + 'tmp/')
jsonfile = df_files.to_json(orient="records")
parsed = json.loads(jsonfile)
with open(path_to_new_folder+'tmp/pymupdf_files.json', 'w') as file:
    json.dump(parsed, file, indent=4)

## process files

In [5]:
df_pdf = df_files[df_files.ImagePDF==False]\
                    .sort_values(by = 'page_count',ascending=True)\
                    .reset_index(drop=True)
df_imagepdf = df_files[df_files.ImagePDF==True]\
                    .sort_values(by = 'page_count',ascending=True)\
                    .reset_index(drop=True)

In [ ]:
df_pdf['path_to_docs'] = df_pdf.progress_apply(lambda x: pymuprocessor.create_markdown(x['orig_filepath'],
                                                    tessdata="location to folder containing language data .../Tesseract-OCR/tessdata",
                                                    folder_location="C:/Users/Prashant/Desktop/work/audit/processing_pymu/" + 'tmp/processing_pdf/',
                                                    filename=os.path.splitext(x['filename'])[0]),axis=1)

  0%|          | 0/4 [00:00<?, ?it/s]

Processing C:/Users/Prashant/Desktop/work/audit/reports\Gulu DLG Report of Auditor General 2019.pdf...
[                                        ] (0/3[=                                       ] ( 1/3[==                                      ] ( 2/37[===                                     ] ( 3/3[====                                    ] ( 4/37[=====                                   ] ( 5/3[======                                  ] ( 6/37[=======                                 ] ( 7/3[========                                ] ( 8/37[=========                               ] ( 9/3[==========                              ] (10/37[===========                             ] (11/3[============                            ] (12/37=[==============                          ] (13/37[===============                         ] (14/3[================                        ] (15/37[=================                       ] (16/3[==================                      ] (17/37[===================    

 50%|█████     | 2/4 [00:05<00:05,  2.86s/it]

]
Processing C:/Users/Prashant/Desktop/work/audit/reports\Gulu DLG Report of Auditor General 2020.pdf...
[                                        ] (0/42[                                        ] ( 1/42[=                                       ] ( 2/4[==                                      ] ( 3/42[===                                     ] ( 4/4[====                                    ] ( 5/42[=====                                   ] ( 6/4[======                                  ] ( 7/42[=======                                 ] ( 8/4[========                                ] ( 9/42[=========                               ] (10/4[==========                              ] (11/42[===========                             ] (12/4[============                            ] (13/42[=============                           ] (14/4[==============                          ] (15/42[===============                         ] (16/4[================                        ] (17/42[=================   

 75%|███████▌  | 3/4 [00:26<00:10, 10.50s/it]

]
Processing C:/Users/Prashant/Desktop/work/audit/reports\Gulu DLG Report of Auditor General 2021.pdf...
[                                        ] (0/42[                                        ] ( 1/42[=                                       ] ( 2/4[==                                      ] ( 3/42[===                                     ] ( 4/4[====                                    ] ( 5/42[=====                                   ] ( 6/4[======                                  ] ( 7/42[=======                                 ] ( 8/4[========                                ] ( 9/42[=========                               ] (10/4[==========                              ] (11/42[===========                             ] (12/4[============                            ] (13/42[=============                           ] (14/4[==============                          ] (15/42[===============                         ] (16/4[================                        ] (17/42[=================   

100%|██████████| 4/4 [00:33<00:00,  9.02s/it]

Processing C:/Users/Prashant/Desktop/work/audit/reports\Gulu DLG Report of Auditor General 2018.pdf...
[                                        ] (0/44[                                        ] ( 1/44[=                                       ] ( 2/4[==                                      ] ( 3/44[===                                     ] ( 4/4[====                                    ] ( 5/44[=====                                   ] ( 6/4[======                                  ] ( 7/44[=======                                 ] ( 8/4[========                                ] ( 9/44[=========                               ] (10/4[==========                              ] (11/4[==========                              ] (12/44[===========                             ] (13/4[============                            ] (14/44[=============                           ] (15/4[==============                          ] (16/44[===============                         ] (17/4[================        

100%|██████████| 4/4 [00:38<00:00,  9.50s/it]

]


process imagepdf using OCR

In [7]:
df_imagepdf['path_to_docs'] = df_imagepdf.progress_apply(lambda x: pymuprocessor.useOCR_create_text(x['orig_filepath'],
                                                    folder_location="C:/Users/Prashant/Desktop/work/audit/processing_pymu/" + 'tmp/processing_pdf/',
                                                    filename=os.path.splitext(x['filename'])[0]),axis=1)

100%|██████████| 1/1 [00:24<00:00, 24.36s/it]


## chunking

In [9]:
df = pd.concat([df_pdf,df_imagepdf],ignore_index=True)

In [18]:
def create_save_chunks(filename,path_to_docs):
    # create chunks
    file_extension = os.path.splitext(os.path.basename(os.path.normpath(path_to_docs)))[1][1:]
    chunks = pymuprocessor.create_chunks(folder_location=path_to_docs,overlap= 10,
                                         chunk_size=800,file_extension=file_extension)
    # save chunks
    with open(path_to_docs+os.path.splitext(filename)[0]+'.chunks.json', 'w') as file:
        json.dump(chunks, file)
    # return chunks filepath
    return path_to_docs+ os.path.splitext(filename)[0] +'.chunks.json'

In [19]:
df['chunks_filepath'] = df.apply(lambda x: create_save_chunks(x['filename'],x['path_to_docs']) 
                                            ,axis=1)

In [ ]:
import json
jsonfile = df.to_json(orient="records")
parsed = json.loads(jsonfile)
with open( "save to folder location", 'w') as file:
    json.dump(parsed, file, indent=4)